## Obtaining camera Intrinsics

In [19]:
import cv2
import numpy as np
import glob
import os

def calibrate_chessboard_images(image_folder, pattern_size=(6, 8), square_size=25.0):
    """
    Calibrate camera given a set of chessboard images.
    
    :param image_folder: Folder containing chessboard images.
    :param pattern_size: Tuple of (number_of_corners_along_width, number_of_corners_along_height).
    :param square_size: Physical size of each chessboard square (in cm, mm, or any consistent unit).
    :return: camera_matrix, dist_coeffs, rvecs, tvecs, error
    """

    # Termination criteria for corner refinement
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    # Prepare 3D object points for a single chessboard
    objp = np.zeros((pattern_size[0] * pattern_size[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)

    # Scale object points by the size of each square
    objp *= square_size

    # Arrays to store object points and image points from all images
    objpoints = []  # 3D points in real-world space
    imgpoints = []  # 2D points in image plane

    images = glob.glob(os.path.join(image_folder, '*_Color.png'))  # or '*.png', etc.
    if not images:
        print(f"No images found in {image_folder}")
        return None, None, None, None, None

    # Track image shape (width, height)
    image_shape = None

    for fname in images:
        img = cv2.imread(fname)
        # If the image can't be read, skip it
        if img is None:
            print(f"Could not read {fname}. Skipping...")
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # We set image_shape if not already set
        if image_shape is None:
            # OpenCV calibrateCamera expects (width, height) order
            image_shape = gray.shape[::-1]

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, pattern_size, None)

        if ret:
            # Refine corner positions to sub-pixel accuracy
            corners_refined = cv2.cornerSubPix(
                gray, corners, (11, 11), (-1, -1), criteria
            )

            objpoints.append(objp)
            imgpoints.append(corners_refined)

            # Draw corners (optional for visualization)
            cv2.drawChessboardCorners(img, pattern_size, corners_refined, ret)
            cv2.imshow('Chessboard Corners', img)
            cv2.waitKey()  # Show each result briefly

    cv2.destroyAllWindows()

    # If we never detected corners, there is nothing to calibrate
    if not objpoints or not imgpoints:
        print("No corners were detected in any image. Calibration aborted.")
        return None, None, None, None, None

    # Perform camera calibration
    ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(
        objpoints,
        imgpoints,
        image_shape,  # Using stored shape
        None,
        None
    )

    # Compute overall reprojection error
    total_error = 0
    for i in range(len(objpoints)):
        imgpoints2, _ = cv2.projectPoints(
            objpoints[i], rvecs[i], tvecs[i], camera_matrix, dist_coeffs
        )
        error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2) / len(imgpoints2)
        total_error += error

    mean_error = total_error / len(objpoints)

    print("Camera matrix:\n", camera_matrix)
    print("Distortion coefficients:", dist_coeffs.ravel())
    print("Mean reprojection error:", mean_error)

    return camera_matrix, dist_coeffs, rvecs, tvecs, mean_error


In [20]:
print(os.getcwd()) # print current working directory

c:\Users\nadil\Desktop\FalconE-F1-Tenth\Labs\lab7_vision_lab


In [21]:
folder = "Resources\calibration"
camera_matrix, dist_coeffs, rvecs, tvecs, mean_error = calibrate_chessboard_images(folder, pattern_size=(6, 8), square_size=25.0)

Camera matrix:
 [[694.71543755   0.         449.37542153]
 [  0.         695.54962013 258.64702588]
 [  0.           0.           1.        ]]
Distortion coefficients: [ 0.14754686  0.19250433 -0.0091839  -0.01212939 -1.70648829]
Mean reprojection error: 0.08230803535365025


## Distance Measurement

In [22]:
import cv2
import numpy as np

# Suppose you have these from prior calibration:
# (Typical shape: fx, 0, cx; 0, fy, cy; 0, 0, 1)
camera_matrix = camera_matrix

# If you have lens distortion, fill this in. Otherwise, assume zero:
dist_coeffs = dist_coeffs  # [k1, k2, p1, p2, k3]

In [23]:
def click_and_get_pixel(image_path):
    """
    Utility function that shows an image and lets you click exactly one pixel.
    Returns (u, v) of the clicked point.
    """
    img = cv2.imread(image_path)
    if img is None:
        raise IOError(f"Cannot read {image_path}")

    clicked_point = []

    def mouse_callback(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            clicked_point[:] = [x, y]  # store in outer list
            print(f"Clicked pixel: (u={x}, v={y})")

    cv2.namedWindow("ClickCone")
    cv2.setMouseCallback("ClickCone", mouse_callback)

    print("Click on the base of the cone, then press ESC.")
    while True:
        cv2.imshow("ClickCone", img)
        key = cv2.waitKey(10)
        if key == 27:  # ESC to break
            break

    cv2.destroyAllWindows()

    if len(clicked_point) == 2:
        return (clicked_point[0], clicked_point[1])  # (u, v)
    else:
        raise ValueError("No click registered.")

# Example usage for known 40cm cone
u_40, v_40 = click_and_get_pixel("Resources/resource/cone_x40cm.png")
print("Pixel for known 40cm cone is:", (u_40, v_40))

Click on the base of the cone, then press ESC.
Clicked pixel: (u=662, v=493)
Pixel for known 40cm cone is: (662, 493)


In [24]:
def estimate_camera_height(u_40, v_40, x_car=0.40, camera_matrix=None):
    """
    Estimate camera height H by assuming:
      - The cone at x_car=0.40m is lying on the ground plane z=0.
      - The camera is pointed horizontally (no pitch).
      - The camera is at some height H above ground.
      - The cone is directly along the optical axis (i.e. no lateral offset).
    Returns approximate H in meters.
    
    In reality, you need a more robust geometry solution or multiple points.
    """
    fx = camera_matrix[0,0]
    cx = camera_matrix[0,2]
    # We'll just do a naive "similar triangles" approach:
    #
    # If the optical axis is horizontal, the ground at distance x_car
    # would appear below the horizon by some amount in image coordinates.
    # The difference (v_40 - cy) is due to camera height H.
    #
    # For small angles, H ~ x_car * ( (v_40 - cy) / fx ).
    #
    # This is not physically perfect. But let's do a rough approximation:
    v_offset = (v_40 - camera_matrix[1,2])  # how many pixels below center
    # We'll say "tan(theta) = H / x_car" ~ v_offset / fx
    # => H = x_car * (v_offset / fx)
    # The sign depends on your coordinate system. We'll assume downward is positive v.
    H = x_car * (abs(v_offset) / fx)

    return H

H_approx = estimate_camera_height(
    u_40, v_40, 
    x_car=0.40,
    camera_matrix=camera_matrix
)
print(f"Approx camera mounting height: {H_approx*100:.1f} cm")

Approx camera mounting height: 13.5 cm


In [25]:
def pixel_to_car(u, v, camera_matrix, H):
    """
    Map pixel (u,v) to (x_car, y_car), assuming:
      - Ground plane is z=0 in 'car' coords.
      - Camera is at (0, H, 0).
      - Optical axis is horizontal (no tilt).
      - +x_car is forward, +y_car is to the left or right (depends on your convention).
    Returns (x_car, y_car).
    
    This is a simplistic pinhole-based approach. Real labs often use a homography 
    or solvePnP with multiple reference points.
    """
    fx = camera_matrix[0,0]
    fy = camera_matrix[1,1]
    cx = camera_matrix[0,2]
    cy = camera_matrix[1,2]

    # In camera coordinates (assuming no tilt):
    # Let's define Z_c = X_car, X_c = Y_car, Y_c = ??? depends on your convention.
    #
    # For a point on the ground, let's do a quick ratio approach:
    #    (v - cy) / fy ~ (Y_c / Z_c)
    #    (u - cx) / fx ~ (X_c / Z_c)
    #
    # But we also know the camera is at height H, so the ground is -H below it
    # in Y_c if the camera's +Y is downward. 
    #
    # A simpler approach is to say the distance from the camera is:
    #   X_car = H * (fx / (v - cy))   if the point is purely in front, ignoring sideways offset
    #
    # That is borrowed from the typical "flat-world assumption" for a downward-facing camera,
    # or you invert your geometry. 
    #
    # We also want y_car if there's a horizontal offset. 
    # Typically: y_car = (u - cx)/fx * x_car. 
    # 
    # The sign of y_car depends on whether you define right or left as positive.

    # For demonstration, let's define:
    dy = (u - cx)  # horizontal pixel offset
    dv = (v - cy)  # vertical pixel offset

    # approximate forward distance:
    x_car = H * (fy / abs(dv)) if dv != 0 else 9999  # big fallback if dv=0
    # approximate lateral distance:
    y_car = x_car * (dy / fx)

    # Depending on your sign conventions, you might invert y_car or x_car
    return (x_car, y_car)


In [26]:
def measure_unknown_cone_distance():
    # 1) Click the base in cone_unknown.png
    u_unknown, v_unknown = click_and_get_pixel("Resources/resource/cone_unknown.png")
    print("Pixel for unknown cone is:", (u_unknown, v_unknown))

    # 2) Use the previously computed height
    print(f"Using camera height H ~ {H_approx:.3f} m from cone_x40cm.png...")

    # 3) Convert that pixel to (x_car, y_car)
    x_car_unknown, y_car_unknown = pixel_to_car(
        u_unknown, 
        v_unknown,
        camera_matrix, 
        H_approx  # from earlier
    )
    print(f"Unknown cone is at x_car={x_car_unknown:.3f} m, y_car={y_car_unknown:.3f} m")

measure_unknown_cone_distance()

Click on the base of the cone, then press ESC.
Clicked pixel: (u=596, v=414)
Pixel for unknown cone is: (596, 414)
Using camera height H ~ 0.135 m from cone_x40cm.png...
Unknown cone is at x_car=0.604 m, y_car=0.128 m
